In [ ]:
import shutil
shutil.rmtree("examplemodel")

In [ ]:
import sys, re
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
import math

Let's try another classification example, with a more complicated classes boundary.<br>We begin with the same model as in example one.

In [ ]:
def model_function(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 1])
    hidden = tf.layers.dense(inputs=input_layer, units=1)
    logits = tf.layers.dense(inputs=hidden, units=2)
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
train_data   = np.asarray( [1, 2, 3, 4, 5, 6, 7, 8], dtype=np.float32)
train_labels = np.asarray( [0, 1, 0, 1, 0, 1, 0, 1], dtype=np.int32)

In [ ]:
estimator = tf.estimator.Estimator(model_fn=model_function, model_dir="examplemodel")
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=8,
    num_epochs=None,
    shuffle=True)

In [ ]:
estimator.train(
  input_fn=train_input_fn,
  steps=1000)



As you can see, its not converging. Again, the classes are too complicated to be expressed with a simple linear binary classifier. We need to introduce the periodicity somehow.<br> The sine function can help us.

In [ ]:

t = np.arange(0., 8., 0.2)

# red dashes, blue squares and green triangles
plt.plot(t, np.sin(t*np.pi/2), 'r--')
plt.show()

In [ ]:
def sineone(x):
    px = tf.multiply(x, tf.constant(1.57079632679))
    return tf.abs(tf.sin(px))

Remember to delete the model before running it again:

In [ ]:
import shutil
shutil.rmtree("examplemodel")

In [ ]:
def model_function(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 1])
    hidden = tf.layers.dense(inputs=input_layer, units=1, activation=sineone)
    logits = tf.layers.dense(inputs=hidden, units=2)
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
estimator = tf.estimator.Estimator(model_fn=model_function, model_dir="examplemodel")
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=8,
    num_epochs=None,
    shuffle=True)

In [ ]:
estimator.train(
  input_fn=train_input_fn,
  steps=10000)

In [ ]:
test_data = [48]
test_data = np.asarray(test_data, dtype=np.float32)
eval_input_fn = tf.estimator.inputs.numpy_input_fn( x={"x":test_data}, num_epochs=1, shuffle=False )
eval_results = estimator.predict(input_fn=eval_input_fn)
print(list(eval_results))

In [ ]:
print(str(estimator.get_variable_names()))
print("WEIGHTS 0"+str(estimator.get_variable_value("dense/kernel")))
print("BIAS 0"+str(estimator.get_variable_value("dense/bias")))
print("WEIGHTS 1"+str(estimator.get_variable_value("dense_1/kernel")))
print("BIAS 1"+str(estimator.get_variable_value("dense_1/bias")))